# Working with malfind output from vol3 on the dc

Get some libraries

In [1]:
import os
import codecs
import pandas as pd

Setup some paths to the files

In [2]:
PATH_TO_FOLDER = '/home/ubuntu/jupyter/cases/stolen_szechuan_sauce/'
CSV_FILE = 'dc-malfind.csv'
PATH_TO_CSV = os.path.join(PATH_TO_FOLDER, CSV_FILE)
print(PATH_TO_CSV)

/home/ubuntu/jupyter/cases/stolen_szechuan_sauce/dc-malfind.csv


Read the file into pandas.

In [3]:
df = None
with codecs.open(PATH_TO_CSV, 'r', encoding='utf-8', errors='replace') as fh:
  df = pd.read_csv(fh, error_bad_lines=False)

print(df.shape)

(15, 12)


In [4]:
df.head()

,TreeDepth,PID,Process,Start VPN,End VPN,Tag,Protection,CommitCharge,PrivateMemory,File output,Hexdump,Disasm
0,0,1292,Microsoft.Acti,0x10500120000,0x1050012ffff,VadS,PAGE_EXECUTE_READWRITE,4,1,Disabled,\n00 00 00 00 00 00 00 00\t........\nb4 26 f3 ...,00 00 00 00 00 00 00 00 b4 26 f3 1c 40 13 00 0...
1,0,1292,Microsoft.Acti,0x10500100000,0x1050010cfff,VadS,PAGE_EXECUTE_READWRITE,1,1,Disabled,\n00 00 00 00 00 00 00 00\t........\n08 00 10 ...,00 00 00 00 00 00 00 00 08 00 10 00 05 01 00 0...
2,0,1292,Microsoft.Acti,0x105001f0000,0x105001fffff,VadS,PAGE_EXECUTE_READWRITE,2,1,Disabled,\n00 00 00 00 00 00 00 00\t........\n5c 67 95 ...,00 00 00 00 00 00 00 00 5c 67 95 04 1e 40 00 0...
3,0,1292,Microsoft.Acti,0x7ff5ff8d0000,0x7ff5ff8dffff,VadS,PAGE_EXECUTE_READWRITE,1,1,Disabled,\n00 00 00 00 00 00 00 00\t........\n78 0d 00 ...,00 00 00 00 00 00 00 00 78 0d 00 00 00 00 00 0...
4,0,1292,Microsoft.Acti,0x7ff5ff8e0000,0x7ff5ff97ffff,VadS,PAGE_EXECUTE_READWRITE,2,1,Disabled,\nd8 ff ff ff ff ff ff ff\t........\n08 00 00 ...,d8 ff ff ff ff ff ff ff 08 00 00 00 00 00 00 0...


Looks like we have some data that might work. There are 15 rows and 11 columns. Let's look a the processes.

In [26]:
df["Process"].value_counts()

Microsoft.Acti    5
spoolsv.exe       4
ServerManager.    3
explorer.exe      2
svchost.exe       1
Name: Process, dtype: int64

Let's look to see if the letters "MZ" show up anywhere.

In [75]:
df[df.stack().str.contains("MZ").any(level=0)]

    

        

    
    
    

,TreeDepth,PID,Process,Start VPN,End VPN,Tag,Protection,CommitCharge,PrivateMemory,File output,Hexdump,Disasm
7,0,3724,spoolsv.exe,0x4afc1f0000,0x4afc25afff,VadS,PAGE_EXECUTE_READWRITE,107,1,Disabled,\n4d 5a 90 00 03 00 00 00\tMZ......\n04 00 00 ...,4d 5a 90 00 03 00 00 00 04 00 00 00 ff ff 00 0...
8,0,3724,spoolsv.exe,0x4afc070000,0x4afc0a8fff,VadS,PAGE_EXECUTE_READWRITE,57,1,Disabled,\n4d 5a 41 52 55 48 89 e5\tMZARUH..\n48 83 ec ...,4d 5a 41 52 55 48 89 e5 48 83 ec 20 48 83 e4 f...
9,0,3724,spoolsv.exe,0x4afc260000,0x4afc283fff,VadS,PAGE_EXECUTE_READWRITE,36,1,Disabled,\n4d 5a 90 00 03 00 00 00\tMZ......\n04 00 00 ...,4d 5a 90 00 03 00 00 00 04 00 00 00 ff ff 00 0...
